In [1]:
import time, datetime
import sys
import subprocess as sp
import os
from pathlib import Path
from sys import exit
#FIXME For copying file it is better to use implemented python function
import shutil
# Conversion of Hartree to kcal per mol
au2kcal = 627.509474
# Conversion eV to Joule
ev2jl = 1.6021766339999E-19
# Conversion of Hartree to electronvolt
from pyscf.data.nist import HARTREE2EV as au2ev
import matplotlib.pyplot as plt

Fontconfig error: Cannot load default config file: No such file: (null)


In [14]:
import pandas.util  # Assuming 'util' is an alias for pandas.util
import pandas as pd
# Read the CSV file

In [15]:
df = pd.read_csv("best_GDB9.csv")
df_smiles=df.iloc[0:10,0:2]

df_smiles

,smiles_key,SMILES
0,gdb 6,C=O
1,gdb 11,CC=O
2,gdb 18,CC(C)=O
3,gdb 23,C#CC#C
4,gdb 24,C#CC#N
5,gdb 25,N#CC#N
6,gdb 26,O=CC#C
7,gdb 27,O=CC#N
8,gdb 28,O=CC=O
9,gdb 35,CCC=O


In [17]:
import os
import time
import numpy as np
import pandas as pd
import subprocess as sp
import re

# Remplace ou échappe les caractères spéciaux dans les noms de fichiers
def sanitize_filename(smiles):
    # Remplacer les caractères spéciaux par des underscores ou d'autres caractères sûrs
    return re.sub(r'[=()#]', '_', smiles)



def ML_calib(smiles_list):
    results = []
    for smiles in smiles_list:
        sanitized_filename = sanitize_filename(smiles.strip())
        # Exécute la commande chemprop_predict pour chaque SMILES
        os.system('chemprop_predict --test_path xTB_' + sanitized_filename +
                  ' --checkpoint_dir /rds/general/user/sv920/home/Databases/SCOP_DB/QMsym_SCOP_ML '
                  '--preds_path xTB_ML_errs_' + sanitized_filename + ' > chemprop_progress.out 2>&1')

        # Lecture des prédictions
        try:
            pred_file = 'xTB_ML_errs_' + sanitized_filename
            if not os.path.exists(pred_file):
                raise FileNotFoundError(f"{pred_file} not found")

            with open(pred_file, 'r') as file:
                errData = file.readlines()

            xtbS1Ind = -1
            xtbT1Ind = -1
            S1errInd = -1
            T1errInd = -1

            # Extraction des indices
            headers = errData[0].strip().split(',')
            for index, val in enumerate(headers):
                if val == 'xtbS1':
                    xtbS1Ind = index
                if val == 'xtbT1':
                    xtbT1Ind = index
                if val == 'S1err':
                    S1errInd = index
                if val == 'T1err':
                    T1errInd = index

            # Calcul des calibrations et FOM pour chaque molécule
            for line in errData[1:]:
                values = line.strip().split(',')
                smiles = values[0]
                S1calib = float(values[xtbS1Ind]) + float(values[S1errInd])
                T1calib = float(values[xtbT1Ind]) + float(values[T1errInd])

                if T1calib == 0 or S1calib == 0:
                    continue

                Sens_FOM = 0 if T1calib > S1calib else np.exp(-(1 - T1calib / S1calib))
                Emit_FOM = 0 if S1calib > 2 * T1calib else np.exp(-(2 - S1calib / T1calib))
                Sens_Good = 1 if 0.95 < T1calib / S1calib < 1 else 0
                Emit_Good = 1 if 1.9 < S1calib / T1calib < 2 else 0

                results.append({
                    'SMILES': smiles,
                    'S1_xTB_ML': S1calib,
                    'T1_xTB_ML': T1calib,
                    'Sens_FOM': Sens_FOM,
                    'Emit_FOM': Emit_FOM,
                    'Sens_Good': Sens_Good,
                    'Emit_Good': Emit_Good
                })

        except Exception as e:
            print(f"Error processing {smiles}: {e}")

    return pd.DataFrame(results)

# Suivi du progrès
with open('progress_ML.out', 'w') as file:
    file.write('Calibrating with ML\n')

startTime = time.time()

# Exécution de la calibration pour tous les SMILES du DataFrame
results_df = ML_calib(df_smiles['SMILES'])

# Écriture des résultats dans un fichier CSV
results_df.to_csv('xTB_ML_calib_results.csv', index=False)

# Enregistrement de la fin du processus
with open('progress_ML.out', 'a') as file:
    file.write('ML calib completed\n')
    file.write('in ' + str(time.time() - startTime) + ' seconds\n')

# Indicateur de fin
with open('done_xTB_ML_calib.txt', 'w') as file:
    file.write('done')


Error processing C=O: xTB_ML_errs_C_O not found
Error processing CC=O: xTB_ML_errs_CC_O not found
Error processing CC(C)=O: xTB_ML_errs_CC_C__O not found
Error processing C#CC#C: xTB_ML_errs_C_CC_C not found
Error processing C#CC#N: xTB_ML_errs_C_CC_N not found
Error processing N#CC#N: xTB_ML_errs_N_CC_N not found
Error processing O=CC#C: xTB_ML_errs_O_CC_C not found
Error processing O=CC#N: xTB_ML_errs_O_CC_N not found
Error processing O=CC=O: xTB_ML_errs_O_CC_O not found
Error processing CCC=O: xTB_ML_errs_CCC_O not found


In [10]:
import os
import time
import numpy as np
import pandas as pd
import subprocess as sp



def ML_calib(smiles_list):
    results = []
    for smiles in smiles_list:
        smiles_filename = smiles.strip()
        # Exécute la commande chemprop_predict pour chaque SMILES
        os.system('chemprop_predict --test_path xTB_' + smiles_filename +
                  ' --checkpoint_dir /rds/general/user/sv920/home/Databases/SCOP_DB/QMsym_SCOP_ML '
                  '--preds_path xTB_ML_errs_' + smiles_filename + ' > chemprop_progress.out 2>&1')

        # Lecture des prédictions
        try:
            with open('xTB_ML_errs_' + smiles_filename, 'r') as file:
                errData = file.readlines()

            xtbS1Ind = -1
            xtbT1Ind = -1
            S1errInd = -1
            T1errInd = -1

            # Extraction des indices
            headers = errData[0].strip().split(',')
            for index, val in enumerate(headers):
                if val == 'xtbS1':
                    xtbS1Ind = index
                if val == 'xtbT1':
                    xtbT1Ind = index
                if val == 'S1err':
                    S1errInd = index
                if val == 'T1err':
                    T1errInd = index

            # Calcul des calibrations et FOM pour chaque molécule
            for line in errData[1:]:
                values = line.strip().split(',')
                smiles = values[0]
                S1calib = float(values[xtbS1Ind]) + float(values[S1errInd])
                T1calib = float(values[xtbT1Ind]) + float(values[T1errInd])

                if T1calib == 0 or S1calib == 0:
                    continue

                Sens_FOM = 0 if T1calib > S1calib else np.exp(-(1 - T1calib / S1calib))
                Emit_FOM = 0 if S1calib > 2 * T1calib else np.exp(-(2 - S1calib / T1calib))
                Sens_Good = 1 if 0.95 < T1calib / S1calib < 1 else 0
                Emit_Good = 1 if 1.9 < S1calib / T1calib < 2 else 0

                results.append({
                    'SMILES': smiles,
                    'S1_xTB_ML': S1calib,
                    'T1_xTB_ML': T1calib,
                    'Sens_FOM': Sens_FOM,
                    'Emit_FOM': Emit_FOM,
                    'Sens_Good': Sens_Good,
                    'Emit_Good': Emit_Good
                })

        except Exception as e:
            print(f"Error processing {smiles_filename}: {e}")

    return pd.DataFrame(results)

# Suivi du progrès
with open('progress_ML.out', 'w') as file:
    file.write('Calibrating with ML\n')

startTime = time.time()

# Exécution de la calibration pour tous les SMILES du DataFrame
results_df = ML_calib(df_smiles)

# Écriture des résultats dans un fichier CSV
results_df.to_csv('xTB_ML_calib_results.csv', index=False)

# Enregistrement de la fin du processus
with open('progress_ML.out', 'a') as file:
    file.write('ML calib completed\n')
    file.write('in ' + str(time.time() - startTime) + ' seconds\n')

# Indicateur de fin
with open('done_xTB_ML_calib.txt', 'w') as file:
    file.write('done')


Error processing C=O: [Errno 2] No such file or directory: 'xTB_ML_errs_C=O'
Error processing CC=O: [Errno 2] No such file or directory: 'xTB_ML_errs_CC=O'
Error processing CC(C)=O: [Errno 2] No such file or directory: 'xTB_ML_errs_CC(C)=O'
Error processing C#CC#C: [Errno 2] No such file or directory: 'xTB_ML_errs_C#CC#C'
Error processing C#CC#N: [Errno 2] No such file or directory: 'xTB_ML_errs_C#CC#N'
Error processing N#CC#N: [Errno 2] No such file or directory: 'xTB_ML_errs_N#CC#N'
Error processing O=CC#C: [Errno 2] No such file or directory: 'xTB_ML_errs_O=CC#C'
Error processing O=CC#N: [Errno 2] No such file or directory: 'xTB_ML_errs_O=CC#N'
Error processing O=CC=O: [Errno 2] No such file or directory: 'xTB_ML_errs_O=CC=O'
Error processing CCC=O: [Errno 2] No such file or directory: 'xTB_ML_errs_CCC=O'


sh: 1: Syntax error: "(" unexpected


3